<a href="https://colab.research.google.com/github/matkubita/StockAIFinanceRLAgent/blob/main/RLAgent_3_Compare_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Setup

In [ ]:
!pip install swig
!pip install wrds
!pip install pyportfolioopt
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3 import A2C, DDPG, PPO
from finrl.config import INDICATORS

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set up environment

In [5]:
base_dir = '/content/drive/MyDrive/ZMUM_FinRL'

trade_data_path = f'{base_dir}/trade_data.csv'

models_paths = {
    "A2C": f"{base_dir}/agent_a2c",
    "DDPG": f"{base_dir}/agent_ddpg",
    "PPO": f"{base_dir}/agent_ppo"
}

# Process data

In [ ]:

def process_data(file_path):
    print(f"--> Wczytywanie danych z: {file_path}")
    df = pd.read_csv(file_path)

    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])

    df = df.sort_values(['date', 'tic']).reset_index(drop=True)

    df.index = df['date'].factorize()[0]

    return df

trade = process_data(trade_data_path)
print(trade.head())

# Set up environment for traiding

In [7]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension

print(f"Liczba akcji: {stock_dimension}")
print(f"Przestrzeń stanów (State Space): {state_space}")

buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, **env_kwargs)

Liczba akcji: 10
Przestrzeń stanów (State Space): 101


# Backtesting

In [10]:
df_account_value = pd.DataFrame()

# MODELS_CLASSES = {"A2C": A2C, "DDPG": DDPG, "PPO": PPO}
MODELS_CLASSES = {"DDPG": DDPG}


print("Rozpoczynam backtesting trzech algorytmów...\n")

for model_name, path in models_paths.items():
    # Sprawdzamy czy plik modelu w ogóle istnieje
    if not os.path.exists(path + ".zip"):
        print(f"Nie znaleziono pliku dla {model_name} w {path}.zip")
        continue

    print(f"Testowanie modelu: {model_name}...")

    try:
        model_class = MODELS_CLASSES[model_name]
        loaded_model = model_class.load(path)

        df_result, _ = DRLAgent.DRL_prediction(
            model=loaded_model,
            environment=e_trade_gym
        )


        if df_account_value.empty:
            df_account_value['date'] = df_result['date']

        df_account_value[model_name] = df_result['account_value']


    except Exception as e:
        print(f" Błąd przy modelu {model_name}: {e}\n")



Rozpoczynam backtesting trzech algorytmów...

Nie znaleziono pliku dla A2C w /content/drive/MyDrive/ZMUM_FinRL/agent_a2c.zip
Testowanie modelu: DDPG...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


hit end!
Nie znaleziono pliku dla PPO w /content/drive/MyDrive/ZMUM_FinRL/agent_ppo.zip


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Compare models

In [11]:
if not df_account_value.empty and len(df_account_value.columns) > 1:
    df_plot = df_account_value.copy()
    df_plot.index = pd.to_datetime(df_plot['date'])
    df_plot = df_plot.drop(columns=['date'])


    plt.figure(figsize=(14, 7))
    colors = {'A2C': '#FF9900', 'DDPG': '#00CC00', 'PPO': '#0066CC'}

    for col in df_plot.columns:
        normalized_value = df_plot[col] / df_plot[col].iloc[0]
        plt.plot(df_plot.index, normalized_value, label=col,
                 color=colors.get(col, 'gray'), linewidth=2.5)

    plt.title('Porównanie skuteczności: A2C vs DDPG vs PPO', fontsize=16)
    plt.xlabel('Data', fontsize=12)
    plt.ylabel('Mnożnik Kapitału (1.0 = Start)', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.axhline(1.0, color='black', linestyle='--', alpha=0.5)
    plt.show()

    print("\n====== PODSUMOWANIE FINANSOWE ======")
    summary_data = []

    for col in df_plot.columns:
        start_val = df_account_value[col].iloc[0]
        end_val = df_account_value[col].iloc[-1]
        profit_usd = end_val - start_val
        profit_pct = (profit_usd / start_val) * 100

        summary_data.append([
            col,
            f"${start_val:,.0f}",
            f"${end_val:,.0f}",
            f"${profit_usd:,.0f}",
            f"{profit_pct:.2f}%"
        ])

    summary_df = pd.DataFrame(summary_data, columns=["Algorytm", "Kapitał Startowy", "Kapitał Końcowy", "Zysk ($)", "Zysk (%)"])

    from IPython.display import display
    display(summary_df)

else:
    print("Brak wyników do wyświetlenia. Upewnij się, że pliki modeli (agent_*.zip) są w folderze ZMUM_FinRL.")


====== PODSUMOWANIE FINANSOWE ======


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,Algorytm,Kapitał Startowy,Kapitał Końcowy,Zysk ($),Zysk (%)
0,DDPG,"$1,000,000","$1,610,695","$610,695",61.07%
